# Packages

In [ ]:
# Data wrangling
import pandas as pd
import polars as pl
import polars.selectors as cs
import numpy as np

# Visualisation
import plotnine as pn
import matplotlib.pyplot as plt
from mizani.formatters import comma_format, custom_format, currency_format, percent_format
from IPython.display import clear_output, display
import matplotlib.font_manager as fm
import matplotlib as mpl
from matplotlib import rc
import plotly.express as px

# Utils
import os
from tqdm.notebook import tqdm
import itertools
import yaml
import warnings
import time
import holidays
import pickle

# Modelling
from sklearn.linear_model import Lasso
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    FunctionTransformer,
)
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_absolute_percentage_error,
    root_mean_squared_error,
)
from sklearn.pipeline import FeatureUnion, make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.feature_selection import VarianceThreshold

import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard

rc('text', usetex=False)

jama_colour = [
    "#374e55",
    "#df8f44",
    "#00a1d5",
    "#b24745",
    "#79af97",
    "#6a6599",
    "#80796b",
]

pd.set_option("display.max.columns", 500)
pd.set_option("display.max.columns", 500)


theme_academic = pn.theme(
    text=pn.element_text(family="Latin Modern Roman"),
    plot_title=pn.element_text(weight="bold", size=14, ha="center"),
    legend_text=pn.element_text(size=9),  # Smaller font for legend items
    panel_background=pn.element_rect(fill="white"),  # Clean white background
    panel_border=pn.element_rect(color="grey", size=0.5),
    axis_ticks=pn.element_line(color="grey"),
    panel_grid_major=pn.element_line(color="grey", size=0.1, alpha=0.3),
    panel_grid_minor=pn.element_line(color="grey", size=0.1, alpha=0.3),
    legend_background=pn.element_rect(fill="white", color=None),
    legend_key=pn.element_rect(fill="white", color=None),
    plot_margin=0.02,
    figure_size=(6, 4),  # Set default figure size (width, height in inches)
)

%matplotlib inline

# Loading the data

In [ ]:
df = pl.read_csv(
    "../0_data/preprocessed/df_final_reduced.csv", try_parse_dates=True
).filter(pl.col("datetime") >= pd.Timestamp("2021-09-01 00:00"))

df.head()

In [ ]:
df.shape

In [ ]:
df

# Missing values

Just forward fill for now.

In [ ]:
df = df.fill_null(strategy="forward")

# Calendar Features

In [ ]:
df = df.with_columns(
    day_of_month=pl.col("datetime").dt.day(),
    day_of_year=pl.col("datetime").dt.ordinal_day(),
    day_of_week=pl.col("datetime").dt.weekday(),
    month=pl.col("datetime").dt.month(),
    hour=pl.col("datetime").dt.hour(),
    year=pl.col("datetime").dt.year(),
)

# Holidays

In [ ]:
# Define the region (Canton of Berne) and the country (Switzerland)
country = "CH"
prov = "ZH"

# Create a list of the regional holidays for the canton of Berne
regional_holidays = holidays.CH(
    years=df["datetime"].dt.year().unique().to_list(), prov=prov
)

In [ ]:
holiday_df = pl.DataFrame(
    {
        "holiday_name": list(regional_holidays.values()),
        "holiday_date": list(regional_holidays.keys()),
    }
).sort("holiday_date")

holiday_df.head()

In [ ]:
df = (
    df.with_columns(date=pl.col("datetime").dt.date())
    .join(holiday_df, how="left", left_on="date", right_on="holiday_date")
    .drop("date")
    .with_columns(holiday_name=pl.col("holiday_name").fill_null("no_holiday"))
)

df.head()

# Column naming convention stipulated by neuralforecast

- ds: datetime column with timestamps
- unique_id: for multivariate forecasting use different string values, for univariate, use one
- y: target variable
- include exogenous predictors as columns to the right

In [ ]:
df = df.rename({"datetime": "ds", "kWh": "y"}).with_columns(unique_id=pl.lit("kWh"))

df.head()

In [ ]:
df = pl.concat(
    [df.select(["unique_id", "ds", "y"]), df.drop(["ds", "unique_id", "y"])],
    how="horizontal",
)

df.head()

# Cyclical Encoding

In [ ]:
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))


def encode_cyclically(column_name, periodicity, table):
    # Create sin and cos encoding
    table = table.with_columns(
        sin_transformer(periodicity)
        .fit_transform(table[column_name])
        .alias(f"{column_name}_sin")
    )

    table = table.with_columns(
        cos_transformer(periodicity)
        .fit_transform(table[column_name])
        .alias(f"{column_name}_cos")
    )
    # Drop the old column
    table = table.drop(column_name)

    return table

In [ ]:
# Dictionary with column name and calendar periodicity
calendar_features = {
    "day_of_month": 31,
    "day_of_year": 365,
    "day_of_week": 7,
    "month": 12,
    "hour": 24,
}

for column_name, periodicity in calendar_features.items():
    df = encode_cyclically(column_name, periodicity, df)

# sklearn Pipeline

In [ ]:
cat_cols = ["holiday_name"]

num_cols = df.select(
    cs.contains(
        "soil_temperature_7_to_28cm",
        "shortwave_radiation",
    )
).columns + ["year"]

manual_cols = df.select(pl.selectors.contains("_cos", "_sin", "is_")).columns + [
    "unique_id",
]

In [ ]:
df.drop(manual_cols + cat_cols + num_cols)

In [ ]:
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        #    ("scaler", StandardScaler())
    ]
)

In [ ]:
categorical_transformer = Pipeline(
    steps=[
        (
            "encoder",
            OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
        ),
    ]
)

In [ ]:
column_transformer = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, num_cols),
        ("categorical", categorical_transformer, cat_cols),
    ],
    remainder="passthrough",
)

In [ ]:
preprocessor = Pipeline(
    steps=[
        ("column_transformer", column_transformer),
        (
            "variance_threshold",
            VarianceThreshold(threshold=0.0),
        ),  # Drops constant columns after transformations
    ]
)

# Hyperparameter Tuning

## Manual test

In [ ]:
df_train = df.filter(
    (pl.col("ds") >= pl.datetime(2021, 9, 1))
    & (pl.col("ds") <= pl.datetime(2022, 8, 31))
).to_pandas()
df_val = df.filter(
    (pl.col("ds") >= pl.datetime(2022, 9, 1))
    & (pl.col("ds") <= pl.datetime(2023, 8, 31))
).to_pandas()

In [ ]:
X_train = df_train.drop(columns=["unique_id", "ds", "y"])
X_val = df_val.drop(columns=["unique_id", "ds", "y"])

In [ ]:
X_train_preprocessed = pd.DataFrame(
    preprocessor.fit_transform(X_train),
    columns=preprocessor.get_feature_names_out(),
)

X_val_preprocessed = pd.DataFrame(
    preprocessor.transform(X_val),
    columns=preprocessor.get_feature_names_out(),
)

In [ ]:
df_train_preprocessed = pd.concat(
    [df_train.filter(["unique_id", "ds", "y"]), X_train_preprocessed], axis=1
)

df_val_preprocessed = pd.concat(
    [df_val.filter(["unique_id", "ds", "y"]), X_val_preprocessed], axis=1
)

In [ ]:
# Define the model
model = Sequential(
    [
        Input(shape=(X.shape[1],)),  # Explicitly define the input shape
        Dense(64, activation="relu"),  # Input layer
        Dense(128, activation="relu"),  # Hidden layer
        Dense(24),  # Output layer for 24-hour predictions
    ]
)

# Compile the model
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Set up Tensorboard
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[tensorboard_callback],
)

# Evaluate on test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

# Predict for new data
predictions = model.predict(X_test)  # Shape will be (n_samples, 24)

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.2720 - mae: 0.4336 - val_loss: 0.1026 - val_mae: 0.2686
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0998 - mae: 0.2673 - val_loss: 0.0891 - val_mae: 0.2543
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0904 - mae: 0.2569 - val_loss: 0.0869 - val_mae: 0.2523
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0885 - mae: 0.2558 - val_loss: 0.0856 - val_mae: 0.2511
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0874 - mae: 0.2541 - val_loss: 0.0855 - val_mae: 0.2510
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0868 - mae: 0.2537 - val_loss: 0.0847 - val_mae: 0.2499
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0856 - mae: 0.2520 - val_loss: 0.0845 - val_mae: 0.2499
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0847 - mae: 0.2514 - val_loss: 0.0846 - val_mae: 0.2500
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0851 - mae:

In [25]:
forecaster = NeuralForecast(models=[model], freq="h")
forecaster.fit(df=df_train_preprocessed, val_size=24 * 7)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MQLoss        | 5      | train
1 | valid_loss   | MQLoss        | 5      | train
2 | padder_train | ConstantPad1d | 0      | train
3 | scaler       | TemporalNorm  | 0      | train
4 | blocks       | ModuleList    | 10.0 M | train
5 | out          | Linear        | 3.0 K  | train
-------------------------------------------------------
10.0 M    Trainable params
9.4 K     Non-trainable params
10.0 M    Total params
40.049    Total estimated model params size (MB)
42        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [26]:
forecaster.predict(futr_df=df_val_preprocessed.drop(columns=["y"]))

ValueError: There are missing combinations of ids and times in `futr_df`.
You can run the `make_future_dataframe()` method to get the expected combinations or the `get_missing_future(futr_df)` method to get the missing combinations.

In [ ]:
df_val_preprocessed.drop(columns=["y"])

,unique_id,ds,numeric__Zurich_shortwave_radiation,numeric__Zurich_soil_temperature_7_to_28cm,numeric__year,categorical__holiday_name_Auffahrt,categorical__holiday_name_Berchtoldstag,categorical__holiday_name_Karfreitag,categorical__holiday_name_Nationalfeiertag,categorical__holiday_name_Neujahrestag,categorical__holiday_name_Ostermontag,categorical__holiday_name_Pfingstmontag,categorical__holiday_name_Stephanstag,categorical__holiday_name_Tag der Arbeit,categorical__holiday_name_Weihnachten,categorical__holiday_name_no_holiday,remainder__day_of_month_sin,remainder__day_of_month_cos,remainder__day_of_year_sin,remainder__day_of_year_cos,remainder__day_of_week_sin,remainder__day_of_week_cos,remainder__month_sin,remainder__month_cos,remainder__hour_sin,remainder__hour_cos
0,kWh,2022-09-01 00:00:00,0.0,19.642000,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.012985e-01,0.97953,-0.871706,-0.490029,-0.433884,-0.900969,-1.000000,-1.836970e-16,0.000000,1.000000
1,kWh,2022-09-01 01:00:00,0.0,19.442000,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.012985e-01,0.97953,-0.871706,-0.490029,-0.433884,-0.900969,-1.000000,-1.836970e-16,0.258819,0.965926
2,kWh,2022-09-01 02:00:00,0.0,19.292000,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.012985e-01,0.97953,-0.871706,-0.490029,-0.433884,-0.900969,-1.000000,-1.836970e-16,0.500000,0.866025
3,kWh,2022-09-01 03:00:00,0.0,19.042000,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.012985e-01,0.97953,-0.871706,-0.490029,-0.433884,-0.900969,-1.000000,-1.836970e-16,0.707107,0.707107
4,kWh,2022-09-01 04:00:00,0.0,18.842000,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.012985e-01,0.97953,-0.871706,-0.490029,-0.433884,-0.900969,-1.000000,-1.836970e-16,0.866025,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8732,kWh,2023-08-30 20:00:00,38.0,17.042000,2023.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-2.012985e-01,0.97953,-0.854322,-0.519744,0.433884,-0.900969,-0.866025,-5.000000e-01,-0.866025,0.500000
8733,kWh,2023-08-30 21:00:00,0.0,17.042000,2023.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-2.012985e-01,0.97953,-0.854322,-0.519744,0.433884,-0.900969,-0.866025,-5.000000e-01,-0.707107,0.707107
8734,kWh,2023-08-30 22:00:00,0.0,16.991999,2023.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-2.012985e-01,0.97953,-0.854322,-0.519744,0.433884,-0.900969,-0.866025,-5.000000e-01,-0.500000,0.866025
8735,kWh,2023-08-30 23:00:00,0.0,16.892000,2023.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-2.012985e-01,0.97953,-0.854322,-0.519744,0.433884,-0.900969,-0.866025,-5.000000e-01,-0.258819,0.965926


In [ ]:
forecaster.make_future_dataframe()

0    2022-08-31 01:00:00
1    2022-08-31 02:00:00
2    2022-08-31 03:00:00
3    2022-08-31 04:00:00
4    2022-08-31 05:00:00
5    2022-08-31 06:00:00
6    2022-08-31 07:00:00
7    2022-08-31 08:00:00
8    2022-08-31 09:00:00
9    2022-08-31 10:00:00
10   2022-08-31 11:00:00
11   2022-08-31 12:00:00
12   2022-08-31 13:00:00
13   2022-08-31 14:00:00
14   2022-08-31 15:00:00
15   2022-08-31 16:00:00
16   2022-08-31 17:00:00
17   2022-08-31 18:00:00
18   2022-08-31 19:00:00
19   2022-08-31 20:00:00
20   2022-08-31 21:00:00
21   2022-08-31 22:00:00
22   2022-08-31 23:00:00
23   2022-09-01 00:00:00
Name: ds, dtype: datetime64[us]

: 

In [26]:
def validation(model, df_val, verbose=True):
    # Validation loop (predict the next 24 hours)
    val_index = 0
    horizon = 24
    val_preds = []

    while val_index + horizon <= df_val.shape[0]:

        # Check for first day after validation period
        if val_index == 0:
            # Get date from D+1
            df_pred_day = df_val.iloc[val_index : val_index + horizon]

            # Make prediction
            y_pred = model.predict(steps=24, exog=df_val.drop(columns=["kWh"]))

            # Save prediction
            val_preds.append(df_pred_day.filter(["kWh"]).join(y_pred))

            # Set D+0 to what was previously D+1
            df_current_day = df_pred_day

        # All other days after first day
        else:
            # Get date from D+1
            df_pred_day = df_val.iloc[val_index : val_index + horizon]

            # Make prediction
            y_pred = model.predict(
                steps=24,
                exog=df_pred_day.drop(columns=["kWh"]),
                last_window=df_current_day["kWh"],
                last_window_exog=df_current_day.drop(columns=["kWh"]),
            )

            # Save prediction
            val_preds.append(df_pred_day.filter(["kWh"]).join(y_pred))

            # Set D+0 to what was previously D+1
            df_current_day = df_pred_day

        val_index += horizon

        if val_index % (24 * 7) == 0:
            print(f"Hour {val_index} of {df_val.shape[0]}")
            clear_output(wait=True)

    y_preds_val = pd.concat(val_preds)

    return y_preds_val

In [27]:
y_preds_val = validation(model=forecaster, df_val=df_val)

Hour 8736 of 8760


In [28]:
fig = px.line(
    y_preds_val.reset_index(),
    x="datetime",
    y=["kWh", "pred"],
    labels={
        "datetime": "Date",
        "value": "Energy Consumption (kWh)",
        "variable": "Series",
    },
    title="Actual vs Predicted Energy Consumption Over Time",
)

# Customize the layout
fig.update_layout(
    template="plotly_white",
    legend=dict(title=""),
    xaxis_title="Date",
    yaxis_title="kWh",
)

In [29]:
def sarimax_trainable(config, eval_option="in-sample"):

    try:
        # Fit the model on the training data
        forecaster = ForecasterSarimax(
            regressor=Sarimax(
                order=(config["p"], config["d"], config["q"]),
                seasonal_order=(config["P"], config["D"], config["Q"], 168),
            )
        )

        forecaster.fit(
            y=df_train["kWh"],
            exog=df_train.drop(columns=["kWh"]),
            suppress_warnings=True,
        )

        if eval_option == "in-sample":
            # Get the Information Criterion
            inf_criterion = forecaster.regressor.get_info_criteria("aic")
            train.report({"loss": inf_criterion})
        elif eval_option == "out-of-sample":
            y_preds_val = validation(forecaster)
            loss = root_mean_squared_error(
                y_pred=y_preds_val["pred"], y_true=y_preds_val["kWh"]
            )
            train.report({"loss": loss})

    except Exception as e:
        tune.report(metric=float("inf"))

In [ ]:
# Need this line for locally defined modules to work with ray
# ray.init(runtime_env={"working_dir": "."}, ignore_reinit_error=True)
np.random.seed(42)

analysis = tune.run(
    tune.with_parameters(
        sarimax_trainable,
        eval_option="out-of-sample",
    ),
    config={
        "p": tune.randint(1, 168),
        "q": tune.randint(1, 168),
        "P": tune.randint(1, 24),
        "Q": tune.randint(1, 24),
        "d": tune.randint(1, 5),
        "D": tune.randint(1, 5),
    },
    metric="loss",
    mode="min",
    name="SARIMAX",
    search_alg=OptunaSearch(),
    time_budget_s=60 * 60 * 1.5,
    num_samples=-1,
    raise_on_failed_trial=False,
    trial_dirname_creator=lambda trial: f"{trial.trainable_name}_{trial.trial_id}",
)

c:\Users\mathi\miniconda3\envs\lapro\lib\site-packages\ray\_private\node.py:1362: ResourceWarning:

unclosed file <_io.TextIOWrapper name='C:\\Users\\mathi\\AppData\\Local\\Temp\\ray\\session_2024-10-30_09-59-24_723195_17952\\logs\\gcs_server.out' mode='a' encoding='utf-8'>

c:\Users\mathi\miniconda3\envs\lapro\lib\site-packages\ray\_private\node.py:1362: ResourceWarning:

unclosed file <_io.TextIOWrapper name='C:\\Users\\mathi\\AppData\\Local\\Temp\\ray\\session_2024-10-30_09-59-24_723195_17952\\logs\\gcs_server.err' mode='a' encoding='utf-8'>

c:\Users\mathi\miniconda3\envs\lapro\lib\site-packages\ray\_private\node.py:1367: ResourceWarning:

unclosed file <_io.TextIOWrapper name='C:\\Users\\mathi\\AppData\\Local\\Temp\\ray\\session_2024-10-30_09-59-24_723195_17952\\logs\\monitor.out' mode='a' encoding='utf-8'>

c:\Users\mathi\miniconda3\envs\lapro\lib\site-packages\ray\_private\node.py:1367: ResourceWarning:

unclosed file <_io.TextIOWrapper name='C:\\Users\\mathi\\AppData\\Local\\Te

2024-10-30 10:10:14,318	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-10-30 10:10:14,421	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/mathi/ray_results/SARIMAX' in 0.1020s.


In [15]:
analysis.dataframe().to_csv("2_SARIMAX_trials.csv", index=False)

NameError: name 'analysis' is not defined

# Model Evaluation